In [11]:
from pymongo import MongoClient

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')

# Nom de la base de données et de la collection
db_name = "hopital"
collection_name = "blood_pressure"

# Fonction pour récupérer la dernière valeur de "value" dans la collection "blood_pressure"
def get_latest_blood_pressure_value():
    db = mongo_client[db_name]
    collection = db[collection_name]
    
    # Trouver le dernier document inséré (trié par timestamp décroissant)
    latest_document = collection.find_one(sort=[("timestamp", -1)])
    
    if latest_document:
        latest_value = latest_document.get("value", None)
        if latest_value:
            print(f"La dernière valeur de 'blood_pressure' est : {latest_value}")
        else:
            print("La dernière valeur de 'blood_pressure' est introuvable.")
    else:
        print("Aucun document trouvé dans la collection 'blood_pressure'.")

# Appeler la fonction pour obtenir et afficher la dernière valeur
get_latest_blood_pressure_value()



La dernière valeur de 'blood_pressure' est : 14.96


In [25]:
import paho.mqtt.client as mqtt
from pymongo import MongoClient
import datetime
import joblib
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Charger le modèle enregistré
model = joblib.load('C:/Users/dell/Downloads/prediction etat de santé.pkl')

# Vérifier si le modèle est bien un arbre de décision
if isinstance(model, DecisionTreeClassifier):

    def predict_health_state(heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature):
        # Convertir les entrées en entiers
        heart_rate = float(heart_rate)
        blood_pressure = float(blood_pressure)
        oxygen_saturation = float(oxygen_saturation)
        respiratory_rate = float(respiratory_rate)
        temperature = float(temperature)

        # Exemple de données d'entrée
        data = np.array([[heart_rate, blood_pressure, oxygen_saturation, respiratory_rate, temperature]])

        # Effectuer une prédiction
        predicted_state = model.predict(data)
        health_prediction = "Normal" if predicted_state[0] == 0 else "Pas Normal"
        print("État de santé prédit:", health_prediction)

        # Afficher les règles de l'arbre de décision
        feature_names = ['heart_rate', 'blood_pressure', 'oxygen_saturation', 'respiratory_rate', 'temperature']

        # Obtenir le chemin de décision pour l'exemple de données d'entrée
        node_indicator = model.decision_path(data)
        leaf_id = model.apply(data)

        # Extraire les règles de décision
        rules = []
        for sample_id in range(len(data)):
            node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]

            print(f"Règles pour l'échantillon {sample_id}:")
            for node_id in node_index:
                if leaf_id[sample_id] == node_id:
                    continue

                # Extraire la règle de décision
                if data[sample_id, model.tree_.feature[node_id]] <= model.tree_.threshold[node_id]:
                    threshold_sign = "<="
                else:
                    threshold_sign = ">"

                rules.append(f"Decision node {node_id}: (X[{feature_names[model.tree_.feature[node_id]]}] {threshold_sign} {model.tree_.threshold[node_id]:.2f})")

        # Afficher les règles extraites
        print("\n".join(rules))

        return health_prediction, rules

else:
    print("Le modèle chargé n'est pas un arbre de décision.")

# Configuration MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')

# Configuration MQTT
mqtt_broker = "192.168.1.40"  # L'adresse IP de votre broker MQTT
mqtt_port = 1883

# Initialiser le dictionnaire pour stocker les valeurs des signaux vitaux
vital_signs_data = {
    "temperature": None,
    "heart_rate": None,
    "spo2": None,
    "breath_rate": None
}

# Initialiser la liste pour stocker les ensembles de données complets
complete_vital_signs_data = []

# Callback pour la connexion
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print(f"Failed to connect to MQTT broker with result code {rc}")

    # S'abonner aux topics spécifiés
    client.subscribe("vital_signs/+/temperature", qos=1)
    client.subscribe("vital_signs/+/heart_rate", qos=1)
    client.subscribe("vital_signs/+/spo2", qos=1)
    client.subscribe("vital_signs/+/breath_rate", qos=1)

# Fonction pour récupérer la dernière valeur de "blood_pressure" dans MongoDB
def get_latest_blood_pressure_value():
    db = mongo_client["hopital"]
    collection = db["blood_pressure"]

    # Trouver le dernier document inséré (trié par timestamp décroissant)
    latest_document = collection.find_one(sort=[("timestamp", -1)])

    if latest_document:
        latest_value = latest_document.get("value", None)
        if latest_value:
            print(f"La dernière valeur de 'blood_pressure' est : {latest_value}")
            return latest_value
        else:
            print("La dernière valeur de 'blood_pressure' est introuvable.")
            return None
    else:
        print("Aucun document trouvé dans la collection 'blood_pressure'.")
        return None

# Callback pour la réception de messages
def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")

    # Analyser la topic pour extraire le nom du signal vital
    topic_parts = msg.topic.split('/')
    signal_type = topic_parts[2]  # e.g., 'temperature'

    if signal_type in vital_signs_data:
        # Stocker la valeur du signal vital dans le dictionnaire
        vital_signs_data[signal_type] = msg.payload.decode()

        # Vérifier si toutes les valeurs des signaux vitaux sont présentes
        if all(value is not None for value in vital_signs_data.values()):
            # Récupérer la dernière valeur de "blood_pressure"
            blood_pressure_value = get_latest_blood_pressure_value()

            # Extraire le numéro de lit
            bed_number_str = topic_parts[1]  # e.g., 'bed1'
            bed_number = int(bed_number_str.replace('bed', ''))

            # Créer un ensemble de données complet
            complete_data = {
                "bed": bed_number,
                "temperature": vital_signs_data["temperature"],
                "heart_rate": vital_signs_data["heart_rate"],
                "spo2": vital_signs_data["spo2"],
                "breath_rate": vital_signs_data["breath_rate"],
                "blood_pressure": blood_pressure_value,
                "timestamp": datetime.datetime.now()
            }

            # Ajouter l'ensemble de données complet à la liste
            complete_vital_signs_data.append(complete_data)

            try:
                # Convertir les valeurs en float pour la prédiction
                val_heart_rate = float(vital_signs_data["heart_rate"])
                val_spo2 = float(vital_signs_data["spo2"])
                val_breath_rate = float(vital_signs_data["breath_rate"])
                val_temperature = float(vital_signs_data["temperature"])

                # Prédire l'état de santé
                health_prediction, rules = predict_health_state(val_heart_rate, blood_pressure_value, val_spo2, val_breath_rate, val_temperature)

                # Afficher le tableau des valeurs des signaux vitaux
                vital_signs_table = [
                    val_heart_rate,
                    blood_pressure_value,
                    val_spo2,
                    val_breath_rate,
                    val_temperature
                ]
                print("Tableau des signaux vitaux:", vital_signs_table)

                # Stocker les données dans MongoDB
                db = mongo_client["hopital"]
                for signal, value in vital_signs_data.items():
                    collection = db[signal]
                    data = {
                        "bed": bed_number,
                        "value": value,
                        "timestamp": datetime.datetime.now()
                    }
                    collection.insert_one(data)
                    print(f"Data inserted into collection '{signal}' in database 'hopital'")

                # Insérer dans la collection etat_patient
                collection = db["etat_patient"]
                patient_data = {
                    "bed": bed_number,
                    "health_prediction": health_prediction,
                    "rules": rules,
                    "timestamp": datetime.datetime.now()
                }
                collection.insert_one(patient_data)
                print("Data inserted into collection 'etat_patient' in database 'hopital'")

                # Réinitialiser le dictionnaire des signaux vitaux pour le prochain ensemble de valeurs
                for key in vital_signs_data:
                    vital_signs_data[key] = None

            except ValueError as e:
                                print(f"Erreur de conversion de la valeur: {e}")

# Configuration du client MQTT
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker, mqtt_port, 60)

# Boucle infinie pour maintenir la connexion MQTT
client.loop_forever()

                      


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\AppData\Local\Temp\ipykernel_18264\3436011793.py:200: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with fea

Connected to MQTT broker
Received `25.83` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `22` from `vital_signs/bed1/breath_rate` topic
La dernière valeur de 'blood_pressure' est : 5.47
État de santé prédit: Pas Normal
Règles pour l'échantillon 0:
Decision node 0: (X[respiratory_rate] > 18.50)
Tableau des signaux vitaux: [-999.0, 5.47, -999.0, 22.0, 25.83]
Data inserted into collection 'temperature' in database 'hopital'
Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'
Received `25.80` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `29` from `vital_signs/bed1/br

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Received `25.81` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `18` from `vital_signs/bed1/breath_rate` topic
La dernière valeur de 'blood_pressure' est : 5.47
État de santé prédit: Pas Normal
Règles pour l'échantillon 0:
Decision node 0: (X[respiratory_rate] <= 18.50)
Decision node 1: (X[blood_pressure] <= 121.00)
Decision node 2: (X[blood_pressure] <= 89.50)
Tableau des signaux vitaux: [-999.0, 5.47, -999.0, 18.0, 25.81]
Data inserted into collection 'temperature' in database 'hopital'
Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Received `25.70` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `18` from `vital_signs/bed1/breath_rate` topic
La dernière valeur de 'blood_pressure' est : 5.01
État de santé prédit: Pas Normal
Règles pour l'échantillon 0:
Decision node 0: (X[respiratory_rate] <= 18.50)
Decision node 1: (X[blood_pressure] <= 121.00)
Decision node 2: (X[blood_pressure] <= 89.50)
Tableau des signaux vitaux: [-999.0, 5.01, -999.0, 18.0, 25.7]
Data inserted into collection 'temperature' in database 'hopital'
Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Received `25.66` from `vital_signs/bed1/temperature` topic
Received `-999` from `vital_signs/bed1/heart_rate` topic
Received `-999` from `vital_signs/bed1/spo2` topic
Received `15` from `vital_signs/bed1/breath_rate` topic
La dernière valeur de 'blood_pressure' est : 5.01
État de santé prédit: Pas Normal
Règles pour l'échantillon 0:
Decision node 0: (X[respiratory_rate] <= 18.50)
Decision node 1: (X[blood_pressure] <= 121.00)
Decision node 2: (X[blood_pressure] <= 89.50)
Tableau des signaux vitaux: [-999.0, 5.01, -999.0, 15.0, 25.66]
Data inserted into collection 'temperature' in database 'hopital'
Data inserted into collection 'heart_rate' in database 'hopital'
Data inserted into collection 'spo2' in database 'hopital'
Data inserted into collection 'breath_rate' in database 'hopital'
Data inserted into collection 'etat_patient' in database 'hopital'


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
